### Chroma

* 오픈소스 백터 데이터베이스
* 개발자의 생산성과 행복에 초점을 맞춤

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 0)

# 텍스트 파일 로드
loader1 = TextLoader("./data/nlp-keywords.txt")
loader2 = TextLoader("./data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

print(len(split_doc1))
print(len(split_doc2))

11
6


### Vector-Store 생성

**from_documents()**
* 문서 리스트로부터 백터 저장소를 생성한다.

**매개변수**
* documents : 백터 저장소에 추가할 문서 리스트(필수)
* collection_name : 생성할 컬렉션 이름 (필수)
* embeddings : 임베딩 함수
* ids : 문서 id 리스트
* persist_directory : 컬렉션을 저장할 디렉토리
* client_settings : Chroma 사용자 설정
* client : 사용자 인스턴스
* collection_metadata : 컬렉션 구성정보

In [ ]:
db = Chroma.from_documents(
    documents=split_doc1, 
    embedding=OpenAIEmbeddings(), 
    collection_name="my_db"
)

In [ ]:
# 저장 경로
DB_PATH="./chroma_db"

# 파일로 저장
persist_db = Chroma.from_documents(
    documents=split_doc1, 
    embedding=OpenAIEmbeddings(), 
    collection_name="my_db",
    persist_directory=DB_PATH
)

In [7]:
persist_db.get()

{'ids': ['ca546da2-16e6-463c-9f37-2e68db93e839',
  'e1507f50-ff10-4487-a836-6593df04c643',
  'f2dc96bd-23d3-4a57-965f-13947fa5240f',
  '97374ff3-c665-4d43-8202-186ee7a73a68',
  '5200ce3f-66a2-4034-890e-55807a8a47ec',
  'b49820d1-bfe5-4537-8e6c-6caa54a9f051',
  'b234246b-07e5-480f-bbe6-4db070ac6410',
  'b2d7d813-9c34-4edf-a5fb-625748a4bd7b',
  'b1680cc9-6e2f-4b11-8b0f-2356bd15d372',
  '63715a7b-504d-499d-a6c7-3399a3112e26',
  '56f9a014-8ad3-4b21-96b8-b9b46fadc789'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

**from texts**
* 텍스트 리스트로 벡터 저장소를 생성

**매개변수**
* texts : 백터 저장소에 추가할 텍스트 리스트(필수)
* collection_name : 생성할 컬렉션 이름 (기본값: _LANGCHAIN_DEFAULT_COLLECTION_NAME)
* embeddings : 임베딩 함수
* ids : 문서 id 리스트
* persist_directory : 컬렉션을 저장할 디렉토리
* client_settings : Chroma 사용자 설정
* client : 사용자 인스턴스
* collection_metadata : 컬렉션 구성정보

In [8]:
db2 = Chroma.from_texts(
    ["안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?", "제 이름은 bear 입니다."],
    embedding=OpenAIEmbeddings()
)

In [9]:
db2.get()

{'ids': ['fd522066-9453-4a70-8c3c-c9c16aa1c045',
  '3fa59a0b-4e3c-444f-b0af-315a705767d4'],
 'embeddings': None,
 'documents': ['안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?', '제 이름은 bear 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**similarity_search**
* Chroma 데이터베이스에서 유사도 검색을 수행한다.
* 주어진 쿼리와 가장 유사한 문서를 반환한다.

매개변수
* query : 검색할 쿼리 텍스트
* K : 반환할 결과의 수, 기본값 4
* filter : 메타데이터로 필터링, 기본값 None
<br>

참고
* 리턴시 점수정보 없이 문서만 반환이된다. 점수정보를 얻고싶으면(similarity_search_with_score())

In [11]:
db.similarity_search("csv에 대해 알려줘", k = 2)

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 

**add_documents()**
* 벡터 스토어에 문서 추가를 할 수 있다.

매개변수
* documents : 벡터 저자소에 추가할 문서 리스트
* **kwargs : 추가 키워드 인자
    * ids : 문서 ID 리스트

In [12]:
from langchain_core.documents import Document

db.add_documents(
    [
        Document(
            page_content="안녕하세요 이번에는 문서를 추가해볼 거에요",
            metadata={"source": "mydata.txt"},
            id="1"
        )
    ]
)

['1']

In [13]:
db.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['안녕하세요 이번에는 문서를 추가해볼 거에요'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**add_texts**
* 텍스트를 임베딩하고 벡터 저장소에 추가

매개변수
* texts : 벡터 젖아소에 추가할 텍스트 리스트
* metadatas : 메타데이터 리스트 (기본값 None)
* ids : 문서 ID 리스트 (기본값 None)
    * 입력하지 않으면 UUID를 사용해 자동으로 생성된다.
    * 기존의 id에 추가하면 덮어씌워지게 된다.

In [17]:
db.add_texts(
    ["이전에 추가한 문서를 덮어씌우기", "기존거는 삭제가 됩니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["1", "2"]
)

['1', '2']

In [19]:
db.get(["2"])

{'ids': ['2'],
 'embeddings': None,
 'documents': ['기존거는 삭제가 됩니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**delete()**

* 벡터 저장소에 지정된 ID의 문서를 삭제 

매개변수
* ids : 삭제할 문서의 ID리스트

In [20]:
db.delete(ids=["1"])

In [21]:
db.get(["1","2"])

{'ids': ['2'],
 'embeddings': None,
 'documents': ['기존거는 삭제가 됩니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**reset_collection()**
* 벡터스토어의 컬렉션을 초기화한다.

In [22]:
# 초기화
db.reset_collection()

In [ ]:
db.get() # 문서 조회

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소를 검색기로 변환
**as_retriever**
* 벡터 저장소를 기반으로 VectorStoreRetriever를 생성한다.

매개변수
* **kwargs: 검색할수에 전달할 키워드 인자
    * search_type : 검색유형("smilarity", "mmr")
    * search_kwargs : 검색함수에 추가로 전달할 인자
        * K : 반환할 문서수
        * score_thresold: 최소 유사도 임계값
        * fetch_k: MMR 알고리즘에 전달할 문서수
        * lambda_mult : MMR 결과의 다양성 조절
        * filter : 문서 메타데이터 필터링

In [24]:
db=Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=OpenAIEmbeddings(),
    collection_name="nlp"
)

In [25]:
# 검색기
retriever = db.as_retriever()

retriever.invoke("Word2Vec 에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 

**검색방식 :mmr**
* k : 반환할 문서수 (기본값 : 4)
* fetch_k : MMRK  알고리즘에 전달할 문서 수 (기본값 : 20)
* lambda_mult : MMR 결과의 다양성 조절 (0~1, rlqhsrkqt 0.5, 0 은 유사도만 고려, 1은 다양성만 고려)

In [26]:
# 다양성이 높은 문서 검색
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k":5, "lambda_mult":0.25, "fetch_k":10}
)

retriever.invoke("Word2Vec 에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의

In [ ]:
# 특정 임계값 이상의 유사도를 가진 문서만 탐색
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.8}
)

retriever.invoke("Word2Vec 에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [36]:
# 메타데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter" : {"source" :"./data/nlp-keywords.txt"}, "k":2}
)

retriever.invoke("ESG 에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트